<a href="https://colab.research.google.com/github/Suba-0408/Communify/blob/main/Sign_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
!pip install opencv-python scikit-learn matplotlib



In [ ]:
import os

DATASET_PATH = '/content/drive/MyDrive/ISL/data'
print("Dataset Path:", DATASET_PATH)


Dataset Path: /content/drive/MyDrive/ISL/data


In [ ]:
import zipfile
import os

# Path to your zip in Drive
zip_path = "/content/drive/MyDrive/ISL.zip"
extract_path = "/content/drive/MyDrive/ISL/data"

# Extract only if not already extracted
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

# Set dataset path
DATASET_PATH = extract_path
print("✅ Dataset ready at:", DATASET_PATH)




✅ Dataset ready at: /content/drive/MyDrive/ISL/data


In [ ]:
import cv2
import numpy as np
import glob

def load_dataset(dataset_path):
    images = []
    labels = []
    label_names = sorted([f for f in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, f))])
    label_map = {label: idx for idx, label in enumerate(label_names)}

    print("Found labels:", label_names)

    for label in label_names:
        folder = os.path.join(dataset_path, label)
        files = glob.glob(os.path.join(folder, "*.*"))  # all files in folder
        print(f"📂 {label} -> {len(files)} files")

        for file in files:
            img = cv2.imread(file)
            if img is not None:
                img = cv2.resize(img, (64, 64))  # Resize to 64x64
                images.append(img)
                labels.append(label_map[label])

    return np.array(images), np.array(labels), label_map

print("Loading dataset...")
images, labels, label_map = load_dataset(DATASET_PATH)
print("✅ Finished loading dataset")
print("Total images loaded:", len(images))





Loading dataset...
Found labels: ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
📂 1 -> 1200 files
📂 2 -> 1200 files
📂 3 -> 1200 files
📂 4 -> 1200 files
📂 5 -> 1200 files
📂 6 -> 1200 files
📂 7 -> 1200 files
📂 8 -> 1200 files
📂 9 -> 1200 files
📂 A -> 1200 files
📂 B -> 1200 files
📂 C -> 1200 files
📂 D -> 1200 files
📂 E -> 1200 files
📂 F -> 1200 files
📂 G -> 1200 files
📂 H -> 1200 files
📂 I -> 1200 files
📂 J -> 1200 files
📂 K -> 1200 files
📂 L -> 1200 files
📂 M -> 1200 files
📂 N -> 1200 files
📂 O -> 1200 files
📂 P -> 1200 files
📂 Q -> 1200 files
📂 R -> 1200 files
📂 S -> 1200 files
📂 T -> 1200 files
📂 U -> 1200 files
📂 V -> 1200 files
📂 W -> 1200 files
📂 X -> 1200 files
📂 Y -> 1200 files
📂 Z -> 1200 files
✅ Finished loading dataset
Total images loaded: 42000


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Normalize images
images = images.astype("float32") / 255.0

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

# One-hot encoding for labels
y_train = to_categorical(y_train, num_classes=len(label_map))
y_test = to_categorical(y_test, num_classes=len(label_map))

print("✅ Data prepared")


✅ Data prepared


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(64,64,3)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(len(label_map), activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 35)             │         4,515 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,629,667 (6.22 MB)

 Trainable params: 1,629,667 (6.22 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32
)


Epoch 1/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 219s 207ms/step - accuracy: 0.8744 - loss: 0.4671 - val_accuracy: 1.0000 - val_loss: 3.8909e-05
Epoch 2/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 206s 196ms/step - accuracy: 0.9945 - loss: 0.0158 - val_accuracy: 1.0000 - val_loss: 2.2913e-06
Epoch 3/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 208s 198ms/step - accuracy: 0.9964 - loss: 0.0096 - val_accuracy: 1.0000 - val_loss: 4.0707e-07
Epoch 4/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 258s 194ms/step - accuracy: 0.9974 - loss: 0.0071 - val_accuracy: 1.0000 - val_loss: 2.3085e-05
Epoch 5/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 258s 191ms/step - accuracy: 0.9976 - loss: 0.0066 - val_accuracy: 1.0000 - val_loss: 7.6369e-07
Epoch 6/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 203s 192ms/step - accuracy: 0.9983 - loss: 0.0054 - val_accuracy: 1.0000 - val_loss: 4.4845e-09
Epoch 7/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 199s 190ms/step - accuracy: 0.9988 - loss: 0.0036 - val_accuracy: 1.0000 - val_loss: 1.9017e-09
Epoch 8/10
1050/1050 ━━━━━━━━━━━━━

In [ ]:
save_path = "/content/drive/MyDrive/ISL_Model.h5"
model.save(save_path)
print("✅ Model saved at:", save_path)


✅ Model saved at: /content/drive/MyDrive/ISL_Model.h5
